In [1]:
# Imports
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch libraries
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
# @title Figure Settings
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/content-creation/main/nma.mplstyle")


In [3]:
# @title Helper Functions

def checkExercise1(A, B, C, D):
  """
  Helper function for checking Exercise 1.

  Args:
    A: torch.Tensor
      Torch Tensor of shape (20, 21) consisting of ones.
    B: torch.Tensor
      Torch Tensor of size([3,4])
    C: torch.Tensor
      Torch Tensor of size([20,21])
    D: torch.Tensor
      Torch Tensor of size([19])

  Returns:
    Nothing.
  """
  assert torch.equal(A.to(int),torch.ones(20, 21).to(int)), "Got: {A} \n Expected: {torch.ones(20, 21)} (shape: {torch.ones(20, 21).shape})"
  assert np.array_equal(B.numpy(),np.vander([1, 2, 3], 4)), "Got: {B} \n Expected: {np.vander([1, 2, 3], 4)} (shape: {np.vander([1, 2, 3], 4).shape})"
  assert C.shape == (20, 21), "Got: {C} \n Expected (shape: {(20, 21)})"
  assert torch.equal(D, torch.arange(4, 41, step=2)), "Got {D} \n Expected: {torch.arange(4, 41, step=2)} (shape: {torch.arange(4, 41, step=2).shape})"
  print("All correct")

def timeFun(f, dim, iterations, device='cpu'):
  """
  Helper function to calculate amount of time taken per instance on CPU/GPU

  Args:
    f: BufferedReader IO instance
      Function name for which to calculate computational time complexity
    dim: Integer
      Number of dimensions in instance in question
    iterations: Integer
      Number of iterations for instance in question
    device: String
      Device on which respective computation is to be run

  Returns:
    Nothing
  """
  iterations = iterations
  t_total = 0
  for _ in range(iterations):
    start = time.time()
    f(dim, device)
    end = time.time()
    t_total += end - start

  if device == 'cpu':
    print(f"time taken for {iterations} iterations of {f.__name__}({dim}, {device}): {t_total:.5f}")
  else:
    print(f"time taken for {iterations} iterations of {f.__name__}({dim}, {device}): {t_total:.5f}")

In [6]:
a = torch.tensor([1, 2, 3])
a

tensor([1, 2, 3])

In [7]:
b = ((1, 2), (3, 6))
torch.tensor(b)

tensor([[1, 2],
        [3, 6]])

In [11]:
one = np.ones([2,4])
one = torch.tensor(one)
one

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=torch.float64)

In [12]:
# The numerical arguments we pass to these constructors
# determine the shape of the output tensor

x = torch.ones(5, 3)
y = torch.zeros(2)
z = torch.empty(1, 1, 5)
print(f"Tensor x: {x}")
print(f"Tensor y: {y}")
print(f"Tensor z: {z}")

Tensor x: tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
Tensor y: tensor([0., 0.])
Tensor z: tensor([[[-5.3203e+36,  7.4409e-43,  0.0000e+00,  0.0000e+00,  0.0000e+00]]])


In [20]:
a = torch.rand(1, 3)
b = torch.randn(2, 3)
c = torch.zeros_like(a)
d = torch.rand_like(c)
print(f"Tensor random: {a}")
print(f"Tensor mean 1 and std 0: {b}")
print(f"Tensor of random like: {c}")
print(f"Tensor of random like: {d}")

Tensor random: tensor([[0.8519, 0.5154, 0.0190]])
Tensor mean 1 and std 0: tensor([[ 0.8546, -0.0234, -1.5640],
        [-1.2728, -0.3340,  1.5390]])
Tensor of random like: tensor([[0., 0., 0.]])
Tensor of random like: tensor([[0.3895, 0.1772, 0.3815]])


In [21]:
import torch
torch.manual_seed(0)

In [22]:
import random
random.seed(0)


In [23]:
import numpy as np
np.random.seed(0)

In [24]:
def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness. NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')

In [25]:
a = torch.arange(0, 10, step=1)
b = np.arange(0, 10, step=1)

c = torch.linspace(0, 5, steps=11)
d = np.linspace(0, 5, num=11)

print(f"Tensor a: {a}\n")
print(f"Numpy array b: {b}\n")
print(f"Tensor c: {c}\n")
print(f"Numpy array d: {d}\n")

Tensor a: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Numpy array b: [0 1 2 3 4 5 6 7 8 9]

Tensor c: tensor([0.0000, 0.5000, 1.0000, 1.5000, 2.0000, 2.5000, 3.0000, 3.5000, 4.0000,
        4.5000, 5.0000])

Numpy array d: [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]



## Exercise 1

In [35]:
def tensor_creation(Z):
  """
  A function that creates various tensors.

  Args:
    Z: numpy.ndarray
      An array of shape (3,4)

  Returns:
    A : Tensor
      20 by 21 tensor consisting of ones
    B : Tensor
      A tensor with elements equal to the elements of numpy array  Z
    C : Tensor
      A tensor with the same number of elements as A but with values ∼U(0,1)
    D : Tensor
      A 1D tensor containing the even numbers between 4 and 40 inclusive.
  """

  A = torch.ones(20, 21)
  B = torch.Tensor(Z)
  C = torch.rand_like(A)
  D = torch.arange(4, 41, step=2)

  return A, B, C, D


# numpy array to copy later
Z = np.vander([1, 2, 3], 4)

# Uncomment below to check your function!
A, B, C, D = tensor_creation(Z)
checkExercise1(A, B, C, D)

All correct


In [43]:
a = torch.ones(5, 3)
b = torch.rand(5, 3)
c = torch.empty(5, 3)
d = torch.empty(5, 3)
mul = torch.mul(a, b, out = c)
add = torch.add(a, b, out= d)
#pointwise operators
print(mul)
print(add)

tensor([[0.3702, 0.7656, 0.7404],
        [0.6731, 0.4856, 0.5747],
        [0.5022, 0.1371, 0.6161],
        [0.1324, 0.7426, 0.2021],
        [0.3367, 0.2629, 0.8947]])
tensor([[1.3702, 1.7656, 1.7404],
        [1.6731, 1.4856, 1.5747],
        [1.5022, 1.1371, 1.6161],
        [1.1324, 1.7426, 1.2021],
        [1.3367, 1.2629, 1.8947]])


In [47]:
#elementwise operators
x = torch.tensor([1, 2, 4, 8])
y = torch.tensor([1, 2, 3, 4])
x + y, x - y, x * y, x / y, x**y

(tensor([ 2,  4,  7, 12]),
 tensor([0, 0, 1, 4]),
 tensor([ 1,  4, 12, 32]),
 tensor([1.0000, 1.0000, 1.3333, 2.0000]),
 tensor([   1,    4,   64, 4096]))

In [53]:
x = torch.rand(3, 3)
print(x)
print('sum: ', x.sum())
print('sum rowwise: ', x.sum(axis=0))
print('sum columnwise: ', x.sum(axis=1))
print('\n')
print(f"Mean value of all elements of x {x.mean()}")
print(f"Mean values of the columns of x {x.mean(axis=0)}")
print(f"Mean values of the rows of x {x.mean(axis=1)}")

tensor([[0.7778, 0.9162, 0.7569],
        [0.5188, 0.9309, 0.2063],
        [0.9716, 0.9886, 0.9776]])
sum:  tensor(7.0447)
sum rowwise:  tensor([2.2682, 2.8357, 1.9409])
sum columnwise:  tensor([2.4509, 1.6560, 2.9378])


Mean value of all elements of x 0.7827451825141907
Mean values of the columns of x tensor([0.7561, 0.9452, 0.6470])
Mean values of the rows of x tensor([0.8170, 0.5520, 0.9793])


## Exercise 2

In [62]:
def simple_operations(a1: torch.Tensor, a2: torch.Tensor, a3: torch.Tensor):
    """
    Helper function to demonstrate simple operations
    i.e., Multiplication of tensor a1 with tensor a2 and then add it with tensor a3

    Args:
        a1: Torch tensor
        Tensor of size ([2,2])
        a2: Torch tensor
        Tensor of size ([2,2])
        a3: Torch tensor
        Tensor of size ([2,2])

    Returns:
        answer: Torch tensor
        Tensor of size ([2,2]) resulting from a1 multiplied with a2, added with a3
    """

    answer = torch.add(torch.matmul(a1, a2), a3)
    return answer



# init our tensors
a1 = torch.tensor([[2, 4], [5, 7]])
a2 = torch.tensor([[1, 1], [2, 3]])
a3 = torch.tensor([[10, 10], [12, 1]])
## uncomment to test your function
A = simple_operations(a1, a2, a3)
print(A)

tensor([[20, 24],
        [31, 27]])


In [66]:
def dot_product(b1: torch.Tensor, b2: torch.Tensor):

    """
    Helper function to demonstrate dot product operation
    Dot product is an algebraic operation that takes two equal-length sequences
    (usually coordinate vectors), and returns a single number.
    Geometrically, it is the product of the Euclidean magnitudes of the
    two vectors and the cosine of the angle between them.

    Args:
        b1: Torch tensor
        Tensor of size ([3])
        b2: Torch tensor
        Tensor of size ([3])

    Returns:
        product: Tensor
        Tensor of size ([1]) resulting from b1 scalar multiplied with b2
    """
    # Use torch.dot() to compute the dot product of two tensors
    product = torch.dot(b1, b2)
    return product


# Computing expression 2:
b1 = torch.tensor([3, 5, 7])
b2 = torch.tensor([2, 4, 8])
## Uncomment to test your function
b = dot_product(b1, b2)
print(b)

tensor(82)


In [70]:
x = torch.arange(0, 10)
print(x)
print(x[-1])
print(x[1:3])
print(x[:-2])


tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor(9)
tensor([1, 2])
tensor([0, 1, 2, 3, 4, 5, 6, 7])


In [72]:
# make a 5D tensor
x = torch.rand(1, 2, 3, 4, 5)

print(f" shape of x[0]:{x[0].shape}")
print(f" shape of x[0][0]:{x[0][0].shape}")
print(f" shape of x[0][0][0]:{x[0][0][0].shape}")

 shape of x[0]:torch.Size([2, 3, 4, 5])
 shape of x[0][0]:torch.Size([3, 4, 5])
 shape of x[0][0][0]:torch.Size([4, 5])


In [83]:
z = torch.arange(12).reshape(2, 6)
print(f'Original:\n{z}')
x = z.flatten()
print(f'flatten to the original elements: \n {x}')
y = x.reshape(3, 4)
print(f'reshape: \n {y}')

Original:
tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])
flatten to the original elements: 
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
reshape: 
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


In [85]:
x = torch.randn(1, 10)
# printing the zeroth element of the tensor will not give us the first number!
print(x)
print(x.shape)
print(f"x[0]: {x[0]}")

tensor([[ 0.5429, -0.5430,  1.3089, -0.1468, -0.7791, -0.5856,  0.0736,  0.9282,
         -0.7914,  0.9528]])
torch.Size([1, 10])
x[0]: tensor([ 0.5429, -0.5430,  1.3089, -0.1468, -0.7791, -0.5856,  0.0736,  0.9282,
        -0.7914,  0.9528])


In [86]:
# Let's get rid of that singleton dimension and see what happens now
x = x.squeeze(0)
print(x.shape)
print(f"x[0]: {x[0]}")

torch.Size([10])
x[0]: 0.5428665280342102


In [88]:
# Adding singleton dimensions works a similar way, and is often used when tensors
# being added need same number of dimensions

y = torch.randn(5, 5)
print(f"Shape of y: {y.shape}")

# lets insert a singleton dimension
y = y.unsqueeze(1)
print(f"Shape of y: {y.shape}")
print(y)

Shape of y: torch.Size([5, 5])
Shape of y: torch.Size([5, 1, 5])
tensor([[[-0.5565, -1.1817, -0.9822, -1.4188,  0.5159]],

        [[-0.7512,  0.2989,  0.0790,  1.9876,  0.4388]],

        [[ 0.3811,  1.7162, -0.0039, -2.0564, -1.5919]],

        [[-2.0217,  0.7896,  1.1842,  0.6712, -1.0036]],

        [[ 0.4756,  0.8377,  0.8374, -1.2132,  0.4991]]])


In [96]:
x = torch.randn(23, 34, 45)
x = x.permute(1, 0, 2)
print(x.shape)
# same but in a different format
# can only swap two at a time
x = x.transpose(2, 1)
x = x.transpose(1, 0)
print(x.shape)


torch.Size([34, 23, 45])
torch.Size([45, 34, 23])


In [99]:
# Create two tensors of the same shape
x = torch.arange(12, dtype=torch.float32).reshape((3, 4))
print(x)
y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
print(y)
# Concatenate along rows
cat_rows = torch.cat((x, y), dim=0)

# Concatenate along columns
cat_cols = torch.cat((x, y), dim=1)

# Printing outputs
print('Concatenated by rows: shape{} \n {}'.format(list(cat_rows.shape), cat_rows))
print('\n Concatenated by colums: shape{}  \n {}'.format(list(cat_cols.shape), cat_cols))

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
tensor([[2., 1., 4., 3.],
        [1., 2., 3., 4.],
        [4., 3., 2., 1.]])
Concatenated by rows: shape[6, 4] 
 tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [ 2.,  1.,  4.,  3.],
        [ 1.,  2.,  3.,  4.],
        [ 4.,  3.,  2.,  1.]])

 Concatenated by colums: shape[3, 8]  
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
        [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
        [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]])


In [100]:
x = torch.randn(5)
print(f"x: {x}  |  x type:  {x.type()}")

y = x.numpy()
print(f"y: {y}  |  y type:  {type(y)}")

z = torch.tensor(y)
print(f"z: {z}  |  z type:  {z.type()}")

x: tensor([-1.0868,  0.1153, -2.7522,  0.1934,  0.7367])  |  x type:  torch.FloatTensor
y: [-1.0867969   0.11530989 -2.752199    0.19335043  0.7367095 ]  |  y type:  <class 'numpy.ndarray'>
z: tensor([-1.0868,  0.1153, -2.7522,  0.1934,  0.7367])  |  z type:  torch.FloatTensor


## Exercise 3

In [110]:
def functionA(my_tensor1, my_tensor2):
  """
  This function takes in two 2D tensors `my_tensor1` and `my_tensor2`
  and returns the column sum of
  `my_tensor1` multiplied by the sum of all the elmements of `my_tensor2`,
  i.e., a scalar.

  Args:
    my_tensor1: torch.Tensor
    my_tensor2: torch.Tensor

  Returns:
    output: torch.Tensor
      The multiplication of the column sum of `my_tensor1` by the sum of
      `my_tensor2`.
  """
  output = my_tensor1.sum(axis=1) * my_tensor2.sum()

  return output

## Implement the functions above and then uncomment the following lines to test your code
print(functionA(torch.tensor([[1, 1], [1, 1]]), torch.tensor([[1, 2, 3], [1, 2, 3]])))


tensor([24, 24])


In [119]:
def functionB(my_tensor):
  """
  This function takes in a square matrix `my_tensor` and returns a 2D tensor
  consisting of a flattened `my_tensor` with the index of each element
  appended to this tensor in the row dimension.

  Args:
    my_tensor: torch.Tensor

  Returns:
    output: torch.Tensor
      Concatenated tensor.
  """
  # TODO flatten the tensor `my_tensor`
  my_tensor = my_tensor.flatten()
  # TODO create the idx tensor to be concatenated to `my_tensor`
  idx_tensor = torch.arange(0, len(my_tensor))
  print(idx_tensor.unsqueeze(1))
  print(my_tensor.unsqueeze(1))
  # TODO concatenate the two tensors
  output = torch.cat([idx_tensor.unsqueeze(1), my_tensor.unsqueeze(1)], axis=1)

  return output

## Implement the functions above and then uncomment the following lines to test your code

print(functionB(torch.tensor([[2, 3], [-1, 10]])))


tensor([[0],
        [1],
        [2],
        [3]])
tensor([[ 2],
        [ 3],
        [-1],
        [10]])
tensor([[ 0,  2],
        [ 1,  3],
        [ 2, -1],
        [ 3, 10]])


In [121]:
def functionC(my_tensor1, my_tensor2):
  """
  This function takes in two 2D tensors `my_tensor1` and `my_tensor2`.
  If the dimensions allow it, it returns the
  elementwise sum of `my_tensor1`-shaped `my_tensor2`, and `my_tensor2`;
  else this function returns a 1D tensor that is the concatenation of the
  two tensors.

  Args:
    my_tensor1: torch.Tensor
    my_tensor2: torch.Tensor

  Returns:
    output: torch.Tensor
      Concatenated tensor.
  """
  # TODO check we can reshape `my_tensor2` into the shape of `my_tensor1`
  if torch.numel(my_tensor1) == torch.numel(my_tensor2):
    # TODO reshape `my_tensor2` into the shape of `my_tensor1`
    my_tensor2 = my_tensor2.reshape(my_tensor1.shape)
    # TODO sum the two tensors
    output = my_tensor1 + my_tensor2
  else:
    # TODO flatten both tensors
    my_tensor1 = my_tensor1.reshape(1, -1)
    my_tensor2 = my_tensor2.reshape(1, -1)
    # TODO concatenate the two tensors in the correct dimension
    output = torch.cat([my_tensor1, my_tensor2], axis=1).squeeze()

  return output
print(functionC(torch.tensor([[1, -1], [-1, 3]]), torch.tensor([[2, 3, 0, 2]])))
print(functionC(torch.tensor([[1, -1], [-1, 3]]), torch.tensor([[2, 3, 0]])))

tensor([[ 3,  2],
        [-1,  5]])
tensor([ 1, -1, -1,  3,  2,  3,  0])
